https://youtu.be/u--UVvH-LIQ

# Выбираем пакет данных для обучения

In [1]:
from datasets import load_dataset

emotion_dataset = load_dataset("emotion")
emotion_dataset

c:\Users\user\.conda\envs\hf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [2]:
emotion_dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [3]:

# Для удобной визуализации можно применить pandas
emotion_dataframe = emotion_dataset["train"].to_pandas()
emotion_dataframe.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [4]:
# Чтобы разобраться что обозначают эти числа можно посмотреть характеристики набора данных
features = emotion_dataset["train"].features
features

{'text': Value('string'),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])}

In [5]:
# можно посмотреть какие соотношения между цифрами и значениями
features["label"].int2str(0)

'sadness'

In [6]:
id2label = {idx:features["label"].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [7]:
# label2id = {features["label"].int2str(idx):idx for idx in range(6)}
# или
label2id = {v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [8]:

label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [9]:
# Проверить сбалансированность набора данных
# Например тут 30% данных помечены как 0(sadnees) а редких мало, и соответственно производительность 

emotion_dataframe["label"].value_counts(normalize=True).sort_index()

label
0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: proportion, dtype: float64

# Токенизация данных
BETR это популярная модель, но существуют более легкие и быстрые модели с такой же точностью и лучше использовать их
https://youtu.be/u--UVvH-LIQ?t=563

In [10]:
from transformers import AutoTokenizer

modelcheckpoint = "microsoft/miniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(modelcheckpoint)

In [11]:
# Пример: данные до токенизации
test1 = emotion_dataset["train"]["text"][:1]
print(test1)
# Пример: токенизированные данные
test2 = tokenizer(emotion_dataset["train"]["text"][:1])
print(test2)

['i didnt feel humiliated']
{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


In [12]:
# До обработки
emotion_dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [13]:
# Хотим применить токенизатор ко всем примерам в наборе данных 
# А в библиотеке наборов данных есть метод "map" который позволяет просто передть функцию и применить ее ко всем примерам
def tokenizer_text(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512) # обрезаем длинные примеры потому что у каждой модели есть ограничение по длинне которое она может обработать

emotion_dataset = emotion_dataset.map(tokenizer_text, batched=True)
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [14]:
# После обработки
emotion_dataset['train'][0]

{'text': 'i didnt feel humiliated',
 'label': 0,
 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

# Решение проблемы дисбаланса классов данных для обучения

In [15]:
# Введем коэфциенты обратно пропорциональные процентному количеству отдельных классов в наборе
emotion_dataframe["label"].value_counts().sort_index()

label
0    4666
1    5362
2    1304
3    2159
4    1937
5     572
Name: count, dtype: int64

In [16]:
len(emotion_dataframe)

16000

In [17]:
calss_weights = (1 - (emotion_dataframe["label"].value_counts().sort_index()/len(emotion_dataframe))).values
calss_weights

array([0.708375 , 0.664875 , 0.9185   , 0.8650625, 0.8789375, 0.96425  ])

In [18]:
# Так как тренажер использует PyTorch все веса должны быть тензорами
import torch

class_weights = torch.from_numpy(calss_weights).float().to("cuda")
class_weights

tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], device='cuda:0')

In [19]:
# https://huggingface.co/docs/transformers/main_classes/trainer
# Если посмотреть в документации то можно увидеть что модель может вычислить потери если указан аргумент "labels"
# Поэтому нужно просто переименовать столбец меток "label" в "labes" и все будет готово

emotion_dataset = emotion_dataset.rename_column("label", "labels")
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [20]:
# emotion_dataset = emotion_dataset.rename_column("input_ids", "labels_ids")
# emotion_dataset

Теперь есть все необходимое для создания обучения

# Создание тренажера

In [21]:
# Создание тренажера

from torch import nn
import torch
from transformers import Trainer

# https://youtu.be/u--UVvH-LIQ?t=798
# Создаем пользовательский тренажер чтобы можно было вычислять потреи по своему усмотрению
class WeightLossTrainer(Trainer):
    def compute_loss(self, model, inputs, num_items_in_batch=None, return_outputs=False):
        # Передача данных в модель
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Извлечение меток
        labels = inputs.get("labels")
        # Определение функции потерь с весами классов которые мы определили сами выше 
        # (изменение данной функции предсмотренно в документации https://huggingface.co/docs/transformers/main_classes/trainer)
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        # вычисление потерь
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

# TODO: В этом надо доплнительно разобраться. Нет понимания

# Сбор всех данных воедино

In [22]:
# Сбор всех данных воедино

# Создаем экземпляр модели
from transformers import AutoModelForSequenceClassification # рассматриваем как задачу классификации последовательностей

model = AutoModelForSequenceClassification.from_pretrained(modelcheckpoint, num_labels=6, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/miniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Создаем метрики
from sklearn.metrics import f1_score # f1_score - это среднее значение точности и полноты, это лучший способ учесть дисбаланс данных

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    return{"f1":f1}

In [24]:
# Последнее что нужно указать это гиперпараметры
from transformers import TrainingArguments

batch_size = 32
# регистрировать потерю обучения в каждой эпохе
logging_steps = len(emotion_dataset["train"]) // batch_size
output_dir = "local_models/minilm_emotion"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3, # Количество эпох обучения
    learning_rate=2e-5, # Скорость обучения
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    logging_steps=logging_steps,
    fp16=True, # Использовать смешанную точность, чтобы ускорить процесс
    push_to_hub=True,
    # Запуск оценки после каждой эпохи ***
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [25]:
# Загружаем свой собственный тренер
trainer = WeightLossTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=emotion_dataset["train"],
    eval_dataset=emotion_dataset["validation"],
    tokenizer=tokenizer
)

C:\Users\user\AppData\Local\Temp\ipykernel_34264\4095014986.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightLossTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightLossTrainer(


# Запуск обучения
https://youtu.be/u--UVvH-LIQ?t=1068

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.290800,0.925761,0.646143
2,0.779100,0.594750,0.862778
3,0.550000,0.500048,0.894129


TrainOutput(global_step=1500, training_loss=0.87331494140625, metrics={'train_runtime': 65.8693, 'train_samples_per_second': 728.716, 'train_steps_per_second': 22.772, 'total_flos': 322136666909568.0, 'train_loss': 0.87331494140625, 'epoch': 3.0})

# Использование обученной модели

In [27]:
from transformers import pipeline

modelcheckpoint_new = "local_models/minilm_emotion"
pipe = pipeline("text-classification", model=modelcheckpoint_new)
pipe

Device set to use cuda:0


In [28]:
pipe("I am wont to hit his car")

[{'label': 'anger', 'score': 0.5640459656715393}]